In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib as plt
import re
%matplotlib inline

In [2]:
globData = pd.read_csv(
    "~/projects/thought_leadership/Files OUTPUT/cciOutGLOBAL.csv",
    low_memory=False)
regData = pd.read_csv(
    "~/projects/thought_leadership/Files OUTPUT/cciOutREGION.csv",
    low_memory=False)
indexCols = ['region','sourceFile','year','quarter','project','jobnr','category',
             'date','table','question','question_sub','base','Response','Total']
globData = globData.set_index(indexCols)
regData = regData.set_index(indexCols)
globData = globData.stack().to_frame().reset_index()
regData = regData.stack().to_frame().reset_index()
cols = indexCols
cols.append('Area')
cols.append('Value')
globData.columns = cols
regData.columns = cols
globData['Country/Region'] = 'Country'
regData['Country/Region'] = 'Region'
cciData = pd.concat([globData,regData])
cciData = cciData.drop_duplicates()

In [3]:
change_category = {'new product': 'new products',
                   'mobile': 'mobile usage',
                   'shopper marketing - ecommerce':'shopper e-marketing'
                  }
cciData['category'] = cciData['category'].map(str.lower)\
    .map(lambda x: change_category[x] if x in change_category else x)

In [4]:
cciData['question_sub'] = cciData['question_sub']\
    .map(lambda x: 'None' if 'by ap' in x or 'by sea' in x or 'by country' in x or 'by region' in x or 'index by' in x else x)

In [5]:
indexCols = ["year","quarter","category","question","question_sub","Response","base"]
cciData = cciData.set_index(indexCols)
cciData = cciData[["Total","Area","Value","Country/Region"]].reset_index()

In [6]:
re_base = re.compile('^(BASE)')
re_summary = re.compile('^(SUM)*(S.D.)*(SE)*(TOP 2 BOX)*(BOTTOM 2 BOX)*(MEAN)*(TOTAL)*(MEDIAN)*(SD)*(AVG)*$')
cciData['stat'] = cciData['Response'].map(str.upper)\
    .map(lambda x: 'summary' if re_base.search(x) or re_summary.search(x) else "response")

In [7]:
ap_list = ["AP","AU","CN","HK","ID","IN","JP","KO","MY","NZ","PH","SG","TH","TW","VN"]
eu_list = ["EU","AT","BE","CH","CZ","DE","DK","EE","ES","FI","FR","GB","GR","HR","HU",
                   "IE","IL","IT","LT","LV","NL","NO","PL","PT","RO","RU","SE","TR","UA",
                   "BG","SK","RS","SI"]
ame_list = ["AME","ZA","AE","EG","PK","SA","MO"]
la_list = ["LA","BR","MX","AR","CO","CL","VE","PE"]
na_list = ["NA.","CA","US"]

def assign_region(x):
    if x in ap_list:
        return 'AP'
    elif x in eu_list:
        return 'EU'
    elif x in ame_list:
        return 'AME'
    elif x in la_list:
        return 'LA'
    elif x in na_list:
        return 'NA.'
    return 'OTHER'

cciData['Region'] = cciData['Area'].map(assign_region)

In [8]:
cciData[cciData['Region'] in ('')]

TypeError: 'in <string>' requires string as left operand, not Series

In [9]:
pd.crosstab(cciData[cciData['Region']!='OTHER'].Area,cciData[cciData['Region']!='OTHER'].year)

year,2011,2012,2013,2014,2015,2016
Area,,,,,,
AE,13549,16906,5851,996,2840,2320
AME,0,0,0,996,2841,2320
AP,13549,16906,5851,996,2841,2320
AR,13549,16906,5851,996,2840,2320
AT,13549,16906,5851,996,2840,2320
AU,13549,16906,5851,996,2840,2320
BE,13549,16906,5851,996,2840,2320
BG,0,6238,5851,996,2840,2320
BR,13549,16906,5851,996,2840,2320


In [10]:
cciData.columns

Index(['year', 'quarter', 'category', 'question', 'question_sub', 'Response',
       'base', 'Total', 'Area', 'Value', 'Country/Region', 'stat', 'Region'],
      dtype='object')

In [22]:
cciData.tail(2)

,year,quarter,category,question,question_sub,Response,base,Total,Area,Value,Country/Region,stat,Region
2642310,2016,Q2,cci,Q16. Which of the following categories best de...,None,S.D.,Base: All Respondents,32674.27,AME,22593.91,Region,summary,AME
2642311,2016,Q2,cci,Q16. Which of the following categories best de...,None,S.D.,Base: All Respondents,32674.27,SEA,24579.73,Region,summary,OTHER


In [19]:
cciData['year'].value_counts()

2012    1043742
2011     814139
2013     369609
2015     189760
2016     160080
2014      64982
Name: year, dtype: int64

In [18]:
cciData['quarter'].value_counts()

Q1    747325
Q3    682744
Q2    620483
Q4    591760
Name: quarter, dtype: int64

In [20]:
cciData['category'].value_counts()

consumer demand          1678514
cci                       607259
new products               61425
global consumer            49770
rising food prices         26649
social media               26528
trust in advt              25498
men & women                24822
fresh foods                22680
multi screen usage         22326
london olympics            16226
shopper e-marketing        12993
holiday shopping            9211
healthy eating              9150
mobile usage                9089
corporate citizenship       8662
loyalty                     8379
education                   6741
otc                         5229
financial status            4026
csr                         3780
online shopping             3355
Name: category, dtype: int64

In [59]:
dict(cciData[cciData['question_sub']!='None']['question'].value_counts())

{'Base: All respondents ': 65,
 'Q16. Level of agreement on corporate social responsibility': 3780,
 'Q18. Active Internet searching - When you learn about NEW PRODUCTS (any item you have not purchased in the past) through the methods below,': 819,
 'Q18. Articles/ content on frequently visited news or lifestyle websites - When you learn about NEW PRODUCTS (any item you have not purchased in the past) through the methods below,': 819,
 'Q18. Attending a public event such as a concert, festival, or sports match - When you learn about NEW PRODUCTS (any item you have not purchased in the past) through the methods below,': 819,
 'Q18. Brand/ Manufacturer web page - When you learn about NEW PRODUCTS (any item you have not purchased in the past) through the methods below,': 819,
 'Q18. Direct mail - When you learn about NEW PRODUCTS (any item you have not purchased in the past) through the methods below,': 819,
 'Q18. Friends/ Family told me about it - When you learn about NEW PRODUCTS (any 

In [41]:
cciData[cciData['question'].str.extract('^(Q44)*') == 'Q44']['question_sub'].value_counts()

 - I cannot afford educational opportunities where I live                             1890
(R8) Family finances (banking location, credit cards)                                 1064
(R11)Managing child care outside the home                                             1064
(R13)Locations for social activities (such as dinner out, movies, concerts, etc)      1064
(R12)Social / activity memberships (such as gym/fitness center, social clubs, etc)    1064
(R10)Child care at home                                                               1064
(R1) Food (consumer packaged goods)                                                   1064
(R7) Pharmaceutical Prescription / OTC Drugs                                          1064
(R9) Insurance                                                                        1064
(R5) Personal Electronics (iPod, mobile phone, etc.)                                  1064
(R2) Health / Beauty (cosmetics, hair products, personal hygiene, etc.)               1064

In [58]:
dict(cciData[cciData['question_sub']!='None']['question'].str.extract('^(Q[0-9][0-9])*').value_counts())

{'Q16': 3780,
 'Q18': 17199,
 'Q19': 15876,
 'Q20': 11340,
 'Q21': 5292,
 'Q22': 15258,
 'Q23': 9658,
 'Q24': 8271,
 'Q25': 32474,
 'Q26': 8351,
 'Q27': 13950,
 'Q28': 1362,
 'Q29': 1008,
 'Q30': 6279,
 'Q31': 12615,
 'Q32': 8211,
 'Q33': 21247,
 'Q34': 17096,
 'Q35': 1190,
 'Q36': 2016,
 'Q37': 504,
 'Q38': 3360,
 'Q39': 392,
 'Q40': 7046,
 'Q41': 392,
 'Q42': 11705,
 'Q43': 6258,
 'Q44': 15722,
 'Q45': 9051,
 'Q46': 11088,
 'Q47': 15288,
 'Q48': 11067,
 'Q49': 3003,
 'Q50': 1295,
 'Q51': 1638,
 'Q52': 336,
 'Q53': 616}

In [60]:
cciData['Response'].value_counts()

Base : Unwtd                                                                                      98539
Base : Wtd(In ' 000)                                                                              98539
Sum                                                                                               89247
None                                                                                              27501
1                                                                                                 25971
Average                                                                                           25278
5                                                                                                 24648
3                                                                                                 24648
6                                                                                                 24648
10                                                              